In [143]:
import os
import pandas as pd
import json

In [156]:
class LoadExcel:
    
    fileName = None
    columnNames = None
    columnNameRowIndex = 0
    filePath = None
    qualifieldFilePath = None
    sheetNames = None
    
    def __init__(self,filePath=None, fileName = None,columnNames = None , sheetNames=None , columnNameRowIndex = 0):
        self.filePath = filePath
        self.fileName = fileName
        self.columnNames = columnNames
        self.columnNameRowIndex = columnNameRowIndex
        self.sheetNames = sheetNames
        if fileName is not None and filePath is not None:
            self.qualifieldFilePath = str(filePath) +"/"+ str(fileName)
            
    def loadFile(self, readsheet):
        fload = None        
        print("Readin sgeet :"+readsheet)
        if isinstance(readsheet, str) :
            fload = pd.read_excel(self.qualifieldFilePath,sheet_name= readsheet)
        return fload
    
    def get_col_name_frm_row(self, df,rowno, startColNo,endColNo):
        a1 = df.iloc[rowno,startColNo:endColNo]
        cls = [str(x) for x in a1.values]
        curnt =None
        for i,x in enumerate(cls):
            curnt = x
            if i >0:
                for j,y in enumerate(cls):
                    if j >0 and y==x and i!=j:
                        cls[j] = y+"_"+str(j)
        return cls
    
    def rename_col(self,df, colNameArray):
        colNameArray = [x.replace(' ', '_') for x in colNameArray if x is not None and isinstance(x,str)]
        xlx1 = df.set_axis(colNameArray, axis=1, inplace=False)
        return xlx1
    
    def get_schema_cols(self,df,colNameArray):
        xlx2 = df[colNameArray]
        return xlx2


class SchemaPrep:
    
    df = None
    cols = None
    schemaNameCol = None
    
    def __init__(self, df=None, cols=None,schemaNameCol=None):
        self.df = df
        self.cols = cols
        self.schemaNameCol = schemaNameCol
        
    def get_unq_schema_names(self,df=None,colNm=None):
        df = df if df is not None else self.df        
        colNm = colNm if colNm is not None else self.schemaNameCol
        rtn = None
        if df is not None and colNm is not None:
            a = df[colNm]
            rtn= [str(x) for x in a.values]            
        return rtn
    
    def create_dot_dicts(self,dct,colAr):
        for x in colAr:
            print()
            
        
    def create_dicts(self,pdct,colAr):        
        for x in colAr:
            if "." not in x:
                pdct[x] = {}
        for key in pdct:
            subs = [y[len(key+"."):] for y in colAr if key+"." in y]
            if len(subs)>0:
                self.create_dicts(pdct[key],subs)
                
    def get_table_objs(self,colAr):  
        dct ={}
        self.create_dicts(dct,colAr)
        return dct
    
    def json_property_template(self,descrp=None,typ=None):
        dct = {
            "description":"" if descrp is None else str(descrp).title(),
            "type":"string" if typ is None else str(typ).lower()
        }
        if descrp is None:
            dct["description"] = " "
        else:
            dct["description"] = str(descrp).title()
            
        if typ is None:
            dct["type"] ="string"
        else:
            if isinstance(typ, str):
                if typ.lower() == "object":
                    dct["properties"]={}
                elif typ.lower() == "array":
                    dct["items"]=self.json_property_template(descrp,"object")
                elif typ.lower() == "ref":
                    dct["$ref"]= {}
                elif typ.lower() == "integer":
                    dct["type"] = "integer"
                elif typ.lower() == "int":
                    dct["type"] = "integer"
                elif typ.lower() == "char":
                    dct["type"] = "string"
            elif isinstance(typ,float):
                dct["type"] = "number"
                    
        return dct
    
    def json_schema_template(self,loc=None,title=None,reg=None, adprop=None,descrp=None):
        dct = {
            "$id": None,
            "$schema": "https://json-schema.org/draft/2020-12/schema",
            "title":None,
            "description":None,
            "type":"object",
            "propertyNames":{
                "pattern":None
            },
            "properties":{
                
            },
            "additionalProperties":False if adprop is None else adprop
            
        }
        dct["$id"] = "".join(loc if loc is not None else "https://localhost/schema/{}")
        dct["title"]="".join(title if title is not None else " Json Scheam design for {}")        
        dct["description"]="".join(descrp if descrp is not None else " This is Description {}")
        dct["propertyNames"]["pattern"]= "".join("^[A-Za-z0-9_]+$" if reg is None else reg)
        if adprop is None or adprop == True:
            dct["additionalProperties"]="true"
        else:
            dct["additionalProperties"]="false"
        return dct
        

class GenerateSchema:
    scma = None
    sp = None
    xlx = None
    scDf = None    
    
    def __init__(self,dct=None):
        self.inps = dct if dct is not None else self.inps
        dct = dct if dct is not None else self.inps
        self.le = LoadExcel(dct['filePath'],dct['fileName'],dct['columnNames'],
                            dct['sheetNames'],dct['columnNameRowIndex'])
        self.sp = SchemaPrep()
    
    def loadFle(self,sheetName=None,rename=None,rowno=None, startColNo=None,endColNo=None):
        sheetName = sheetName if sheetName is not None else self.inps['sheetName']
        rename = rename if rename is not None else self.inps['rename']   
        
        xlx = self.le.loadFile(sheetName if sheetName is not None else "Sheet1") 
        print(xlx.columns)
        if rename is True:            
            rowno = rowno if rowno is not None else self.inps['rowno'] 
            startColNo = startColNo if startColNo is not None else self.inps['startColNo']
            endColNo = endColNo if endColNo is not None else self.inps['endColNo']
            
            clns = self.le.get_col_name_frm_row(xlx,rowno,startColNo,endColNo)
            xlx2 = self.le.rename_col(xlx, clns)
            print("--------")
            print(xlx2.columns)
            self.xlx = xlx2
            print("--------")
            print(self.xlx.columns)
            self.xlx.head(5)
        else:
            self.xlx = xlx
        return self.xlx
    
    def optaion_schema_frame(self, schemaCols=None):
        # create sub Data Frame for selecting columns which have schema prep columns
        schemaCols = schemaCols if schemaCols is not None else self.inps['schemaCols']
        print(schemaCols)
        print(self.xlx.columns)
        j1 = self.le.get_schema_cols(self.xlx,schemaCols)
        self.sp.df = j1
        self.scDf = j1
    
    def generate_table_name_object(self,schemaNmsCol=None):
        schemaNmsCol  = schemaNmsCol if schemaNmsCol is not None else self.inps['schemaNmsCol']       
        aa = self.sp.get_unq_schema_names(self.sp.df,schemaNmsCol)[1:]
        dct =self.sp.get_table_objs(aa)
        return dct
    
    def generate_table(self):
        loc="https://localhost/schema/product.schema.json",
        title="Product Final NoSQL JSON Schema",
        reg="^[A-Za-z0-9_]+$", 
        adprop=False,
        descrp="This is Final Product Master Attribute Schema \
                which is hosted for Keeping Data From Final Kafka Product Topic"
        jsn = self.sp.json_schema_template(loc,title,reg,adprop,descrp)
        return jsn
        
    def generate_table_prperties(self, tables,jsn,mtchToken=None):
        d1 =None
        currentItrKey = None
        for key, value in tables.items():           
            jsn["properties"][key] = self.sp.json_property_template(key,"object")
            d1 = self.scDf[self.scDf.Schema_8 == key]
           
            for row in d1.itertuples():
                #print(row.Field)  print(row.Length) print(row.Type) print(row.Comment)
                entryType = row.Data_Type_10
                if row.CustomType is not None and isinstance(row.CustomType,str) == True and row.CustomType.lower() == 'array':
                    entryType = 'array'
                   
                jsn["properties"][key]["properties"][row.Field] = self.sp.json_property_template(row.Comment,entryType)
                
            if mtchToken is not None:
                d2 = self.scDf[self.scDf.Schema_8 == mtchToken+"."+key]
                for row in d2.itertuples():
                    entryType = row.Data_Type_10
                    if row.CustomType is not None and isinstance(row.CustomType,str) == True and row.CustomType.lower() == 'array':
                        entryType = 'array'
                    jsn["properties"][key]["properties"][row.Field] = self.sp.json_property_template(row.Comment,entryType)
            print(value)           
            print("------------>>")
            if len(value)>0:
                print("Has child tables")
                mtchToken1 = key
                if mtchToken is not None:
                    mtchToken1 = mtchToken+"."+key               
                for k, v in value.items():
                    print("Generating Schema --- Start ---mtchToken :: "+ mtchToken1)
                    self.generate_table_prperties(value,jsn["properties"][key],mtchToken1)       
        print("Generating Schema --- End ---")
        return jsn
    
    def runfx(self, filePath = None,fileName = None):
        self.loadFle()
        print(self.xlx.size)
        self.xlx.head(2)
        self.optaion_schema_frame()
        tables = self.generate_table_name_object()  
        jsn = self.generate_table()
        self.generate_table_prperties(tables,jsn)
        if filePath is not None and fileName is not None:
            with open(filepath+"/"+fileName+".json",'w') as f:
                json.dump(jsn,f)
        else:
            print()        
        return jsn


In [157]:
inps = {
        "filePath" : "/samplefile",
        "fileName" : "kfk.xlsx",
        "columnNames" : ["Source Topic",
            "Schema","Attribute Name","Data Type","Length",
            "Decimal","Source To Target Mapping/Transformation Logic",
            "Topc","Schema","Field","Data Type","Length","Decimal","Comment","CustomType"],
        "columnNameRowIndex" : 0,
        "schemaCols" : ["Schema_8", "Field", "Data_Type_10", "Length","Decimal", "Comment","CustomType"],
        "sheetNames":["Sheet1","Sheet2"],
        "rename":True,
        "rowno":0, 
        "startColNo":0,
        "endColNo":15,
        "schemaNmsCol":"Schema_8",
        "sheetName":"Sheet1"
    }
gs = GenerateSchema(inps)
tables = gs.runfx()
print(json.dumps(tables))

Readin sgeet :Sheet1
Index(['Source: Kafka', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Middleware', 'Target:FINAL Product Topic', 'Unnamed: 8',
       'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14'],
      dtype='object')
--------
Index(['Source_Topic', 'Schema', 'Attribute_Name', 'Data_Type', 'Length',
       'Decimal', 'Source_To_Target_Mapping.Transformation_Logic', 'Topc',
       'Schema_8', 'Field', 'Data_Type_10', 'Length_11', 'Decimal_12',
       'Comment', 'CustomType'],
      dtype='object')
--------
Index(['Source_Topic', 'Schema', 'Attribute_Name', 'Data_Type', 'Length',
       'Decimal', 'Source_To_Target_Mapping.Transformation_Logic', 'Topc',
       'Schema_8', 'Field', 'Data_Type_10', 'Length_11', 'Decimal_12',
       'Comment', 'CustomType'],
      dtype='object')
225
['Schema_8', 'Field', 'Data_Type_10', 'Length', 'Decimal', 'Comment', 'CustomType']
Index(['Source_Topic', 'Schema', 'Attr